<a href="https://colab.research.google.com/github/ssreeramj/langgraph-tutorials/blob/main/blog-generator/blog_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install -qU langchain langgraph openai langchain-groq langchain-openai duckduckgo-search langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Imports

In [ ]:
# Standard library imports
import os
import random
import requests

# Third-party imports
from google.colab import userdata
from IPython.display import Image, display
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeColors
from langchain_core.tools import StructuredTool, tool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langgraph.graph import END, MessageGraph, StateGraph
from langgraph.prebuilt import ToolInvocation
from langgraph.prebuilt.tool_executor import ToolExecutor
from typing import Annotated, Sequence, TypedDict, Union, Literal, List

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from pprint import pprint

# Env variables

In [ ]:
# Langsmith tracing
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
# os.environ["LANGCHAIN_PROJECT"] = "blog-generator-from-chat"

In [ ]:
if "LANGCHAIN_TRACING_V2" in os.environ:
    del os.environ["LANGCHAIN_TRACING_V2"]
    del os.environ["LANGCHAIN_ENDPOINT"]
    del os.environ["LANGCHAIN_API_KEY"]
    del os.environ["LANGCHAIN_PROJECT"]

In [ ]:
# Groq model
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

## Extract contents from the chat

In [ ]:
CHAT_URL = "https://chatgpt.com/share/1d25bae0-fa33-4445-8276-b009a9bc28e1"

In [ ]:
import requests
from bs4 import BeautifulSoup
import json


def log_error(error_message):
    """
    Utility function to log errors.
    """
    print(f"Error: {error_message}")


def get_chat_url():
    """
    Function to get the chat URL from the user.
    """
    url = input("Please enter the ChatGPT public chat URL: ")
    return url

def fetch_html_content(url):
    """
    Function to fetch HTML content from the provided URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        return response.text
    except requests.exceptions.RequestException as e:
        log_error(f"Error fetching the HTML content: {e}")
        return None

def extract_json_from_script(html_content):
    """
    Function to extract and parse JSON content from script tags.
    """
    soup = BeautifulSoup(html_content, 'lxml')
    script_elements = soup.find_all("script")

    for script_element in script_elements:
        script_content = script_element.string
        if script_content:
            try:
                json_content = json.loads(script_content)
                return json_content
            except json.JSONDecodeError:
                continue
    log_error("No valid JSON content found in script tags.")
    return None

def extract_messages(json_data):
    """
    Function to extract chat messages from the JSON data in sequence.
    """
    messages = []

    conversation = json_data.get("props", {}).get("pageProps", {}).get("serverResponse", {}).get("data", {}).get("linear_conversation", [])
    title = json_data.get("props", {}).get("pageProps", {}).get("serverResponse", {}).get("data", {}).get("title", 'Empty Title')

    for message_data in conversation:
        message = message_data.get('message', {})
        author = message.get("author", {}).get("role", "unknown")
        content = message.get('content', {})
        parts = content.get('parts', [])

        for part in parts:
            if part and part != "Original custom instructions no longer available":
                messages.append((author, part))

    return title, messages

def clean_and_format_messages(messages):
    """
    Function to clean and format messages for further processing.
    """
    cleaned_messages = []
    for author, message in messages:
        cleaned_messages.append({
            "role": "User" if author == "user" else "AI",
            # "content": "\n[User]: " + message.strip() if author == "user" else "\n\n[AI Response]: " + message.strip()
            "content": message.strip(),
        })
    return cleaned_messages

def get_full_conversation(messages) -> str:
    """
    Function to get the full conversation from the extracted messages.
    """
    full_conversation = ""
    for author, message in messages:
        full_conversation += f"{'expert-1' if author == 'user' else 'expert-2'}: {message}\n\n"
    return full_conversation


html_content = fetch_html_content(CHAT_URL)
json_data = extract_json_from_script(html_content)
title, raw_messages = extract_messages(json_data)
cleaned_messages = clean_and_format_messages(raw_messages)
full_conversation = get_full_conversation(messages=raw_messages)

print(f"Title: {title}")
print(f"Messages: {full_conversation[:300]}" )

Title: Social Media Benefits Explained
Messages: expert-1: When I post something on any social media platform, people say that you get followers XYZ. But, I'm trying to actually think what you get. Like, sometimes I think that when I post something, I don't... Like, right now on LinkedIn, I have 3300 followers. And I try to post something, and the


In [ ]:
print(get_full_conversation(messages=raw_messages[:2]))

expert-1: When I post something on any social media platform, people say that you get followers XYZ. But, I'm trying to actually think what you get. Like, sometimes I think that when I post something, I don't... Like, right now on LinkedIn, I have 3300 followers. And I try to post something, and then I think that what number of followers will this get. Like, I am putting gemstones of, like, material over here, and then only a few people will watch, and nothing will happen about it. I'll get very little followers. I should maybe save content for large follower base. But then I think, on the other hand, that I'm getting something. Like, I don't know what. But, like, when people watch my content, or read my content, then something must be happening. Like, I must be gaining something. So, can you use your knowledge of social media algorithms and technically explain me what am I gaining?
expert-2: Posting on social media, particularly platforms like LinkedIn, can yield various benefits beyo

# LLMs

In [ ]:
model = ChatGroq(model="llama3-8b-8192", max_tokens=8000)
long_context_llm = ChatOpenAI(model="gpt-4o")

In [ ]:
for m in all_messages:
    print(model.get_num_tokens(m))


1115
1127
1204
940
1196
1013
1566
1418
36


In [ ]:
system = "Can you provide a brief summary of the given text? The summary should cover all the key points \
and main ideas presented in the original text, while also condensing the information into a concise and easy-to-understand \
format. Please ensure that the summary includes relevant details and examples that support the main ideas, while avoiding \
any unnecessary information or repetition. The length of the summary should be appropriate for the length and complexity \
of the original text, providing a clear and accurate overview without omitting any important information. Just directly \
give the summary. Do not start with anything else."

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{conversation}\n\nHere is a brief summary of the original text:")
])

summary_llm = chat_prompt | model

resp = summary_llm.invoke({"conversation": all_messages[0]})
print(resp.content)

When posting on social media, people often focus on gaining followers, but there are many benefits beyond just follower growth. The social media algorithm prioritizes content from connections, followed accounts, and relevant content based on user engagement, increasing visibility and reach. Consistent posting leads to engagement and interaction, which can foster relationships and position you as a thought leader. Other benefits include building your personal brand, growing your network, gaining data and insights, content longevity, professional development, SEO benefits, and community building. While it's tempting to save content for a larger follower base, consistent posting is key to growing that base. By sharing high-quality content regularly, you're building your professional presence and future opportunities.


In [ ]:
resps = summary_llm.batch([{"conversation": m} for m in all_messages])
all_resps = '\n'.join([r.content for r in resps])

In [ ]:
class BlogOutlineOutput(BaseModel):
    headline: str = Field(..., description="The headline of the blog")
    index_list: List[str] = Field(..., description="List of table of contents for the blog")

structured_outline_llm = model.with_structured_output(BlogOutlineOutput)

system = "You are an expert in writing the table of contents of a blog by reading the content of a chat. \
You will be given a summaries of a conversation between human and an AI. You will be working with a reviewer to generate \
the table of contents. Take a look at the Feedback if any, the summaries of the conversation
Generate a catchy headline for the blog and a list of values having the table of contents of the blog in proper order. \
Just to give an example, you might want to start with an introduction section, end with a conclusion section, and have the \
table of contents in between in proper order. The table of contents should be short and concise not exceeding more than 10 \
words."

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", "{summaries}")
])

blog_outline_llm = chat_prompt | structured_outline_llm


# response = blog_outline_llm.invoke({"conversation": all_messages})
# print(response)

In [ ]:
ol = blog_outline_llm.invoke({"summaries": all_resps})
print("Headline: ", ol.headline)
for v in ol.index_list:
    print(v)

Headline:  When posting on social media, you gain more than just followers
Benefits of posting on social media
Technical benefits of posting on social media
Increasing online presence and influence
Understanding graph centrality
Strategies for optimizing social media presence
Building strategic partnerships
Optimizing network for maximum influence and reach
Horizontal network expansion strategy


In [ ]:
# outline reviewer llm

system = "As a reviewer, you are tasked with evaluating a table of contents created by an assistant. \
This table of contents is based on summaries of conversations between a user and a bot. Please provide your detailed \
feedback on the table of contents in bulleted points, considering the following aspects:\n\
Clarity: Are the headings and subheadings clear and easy to understand?\n\
Conciseness: Does the table of contents avoid unnecessary detail while still conveying the main points?\n\
Organization: Is the structure logical and well-organized?\n\
Relevance: Do the headings and subheadings accurately reflect the content of the summaries?\n\
Completeness: Does the table of contents cover all the key topics discussed in the conversation?\n\
Your feedback will help improve the quality and usability of the table of contents."

reviewer_prompt = ChatPromptTemplate.from_template("{system}\n\nSummaries of Conversation:{summaries}\n\n\
Table of Contents:\n{table_of_contents}")
# print(reviewer_prompt.format(**{"system": system, "summaries": all_resps, "table_of_contents": '\n'.join(ol.index_list)}))

reviewer_llm = reviewer_prompt | model

r = reviewer_llm.invoke({"system": system, "summaries": all_resps, "table_of_contents": '\n'.join(ol.index_list)})
print(r.content)

Here is my feedback on the table of contents:

**Clarity:**
The headings and subheadings are generally clear and easy to understand. However, some of the titles could be more concise and specific. For example, "Benefits of posting on social media" could be shortened to "Social Media Benefits" and "Strategies for optimizing social media presence" could be shortened to "Social Media Optimization Strategies".

**Conciseness:**
The table of contents is generally concise and to the point. However, some of the headings are still a bit vague, and it would be helpful to have more specific and descriptive titles. Additionally, some of the headings could be combined or reorganized to make the table of contents more logical and streamlined.

**Organization:**
The structure of the table of contents is generally logical and well-organized. However, some of the headings could be reorganized to create a more cohesive and flowing structure. For example, the section on "Understanding graph centrality" 

## Outline generator

In [ ]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    description: str = Field(..., title="Content of the subsection")

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.description}".strip()


class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    description: str = Field(..., title="Content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            f"### {subsection.subsection_title}\n\n{subsection.description}"
            for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{self.description}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the Wikipedia page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the Wikipedia page.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


refine_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert blog writer. You have gathered information from experts. Now, you are writing an outline of blog article. \
You need to make sure that the outline is comprehensive and specific. \
Topic you are writing about: {topic} """,
        ),
        (
            "user",
            "Draft an outline based on the conversations with subject-matter experts:\n\nConversations:\n\n{conversations}\n\nWrite the outline for the blog:",
        ),
    ]
)

# Using turbo preview since the context can get quite long
refine_outline_chain = refine_outline_prompt | long_context_llm.with_structured_output(Outline)

refined_outline = refine_outline_chain.invoke(
    {
        "topic": title,
        "conversations": full_conversation,
    }
)


In [ ]:
print(refined_outline.as_str)

# Social Media Benefits Explained

## Introduction

Introduce the topic of social media benefits, highlighting the importance of understanding the deeper impacts beyond just follower counts.

## Visibility and Reach

Discuss how consistent posting increases visibility and reach on platforms like LinkedIn, emphasizing the role of algorithms.

### Algorithm Prioritization

Explain how social media algorithms prioritize content based on visibility and engagement.

### Future Opportunities

Detail how increased visibility can lead to future connections, collaborations, and opportunities.

## Engagement and Interaction

Explore the importance of engagement and interaction on social media and how it extends reach.

### Algorithmic Favorability

Describe how algorithms favor posts with high engagement, pushing them to a wider audience.

### Building Relationships

Explain how engagement helps foster relationships and positions you as a thought leader.

## Brand Building and Authority

Highlig

In [ ]:
# Initialize the recursive text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100,
                                               separators=["expert-1", "expert-2", "\n\n", "\n", '.'])

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# Chunk the long text
chunks = text_splitter.split_text(full_conversation)

In [ ]:
len(chunks)
print(chunks[52])

**Steps:**
- **LinkedIn Groups:** Join groups that are specific to different roles (e.g., groups for designers, engineers, marketers).
- **Industry Associations:** Participate in industry associations and forums that cater to a wide range of professionals.

### **7. Host Cross-Functional Events**

**Objective:** Position yourself as a connector and leader in your industry.

**Steps:**
- **Webinars and Panels:** Host webinars or panel discussions that bring together experts from different roles. Promote these events widely.
- **Workshops:** Organize workshops or brainstorming sessions on interdisciplinary topics, inviting diverse professionals.

### **8. Leverage Mutual Connections**

**Objective:** Utilize your existing network to facilitate introductions.


In [ ]:
from langchain_community.vectorstores import SKLearnVectorStore

In [ ]:
vectorstore = SKLearnVectorStore.from_texts(
    chunks,
    embedding=embeddings,
)
retriever = vectorstore.as_retriever(k=7)

In [ ]:
retriever.invoke("What's the point of expaning network?")

[Document(metadata={'id': 'e36e7591-c574-434d-8d9e-c7529d355a0d'}, page_content='expert-1: Go deep into optimize your network part.'),
 Document(metadata={'id': 'fed212ef-70e6-4eb6-a0a5-4ad4f8a199ce'}, page_content="expert-2: Sure, let's delve deeper into optimizing your network. Optimizing your network involves strategic actions to enhance your position and influence within the social graph. Here are the detailed steps and strategies to achieve this:\n\n### **Optimizing Your Network**\n\n#### **1. Expand Strategically**\n\n**Objective:** Increase your betweenness and eigenvector centrality by connecting with key individuals who can act as bridges between different segments of your network.\n\n**Steps:**\n\n1. **Identify Key Connectors:**\n   - **Use Tools:** Platforms like LinkedIn Sales Navigator can help identify individuals who are highly connected within specific industries or circles.\n   - **Observe Engagement:** Look for people who frequently interact with various groups, espec

In [ ]:
print(refined_outline.sections[1].as_str)

## Visibility and Reach

Discuss how consistent posting increases visibility and reach on platforms like LinkedIn, emphasizing the role of algorithms.

### Algorithm Prioritization

Explain how social media algorithms prioritize content based on visibility and engagement.

### Future Opportunities

Detail how increased visibility can lead to future connections, collaborations, and opportunities.


In [ ]:
class SubSection(BaseModel):
    subsection_title: str = Field(..., title="Title of the subsection")
    content: str = Field(
        ...,
        title="Full content of the subsection.",
    )

    @property
    def as_str(self) -> str:
        return f"### {self.subsection_title}\n\n{self.content}".strip()


class BlogSection(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    content: str = Field(..., title="Full content of the section")
    subsections: Optional[List[Subsection]] = Field(
        default=None,
        title="Titles and descriptions for each subsection of the blog.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return (
            f"## {self.section_title}\n\n{self.content}\n\n{subsections}".strip()
        )


section_writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert blog writer. Complete your assigned blog section from the following outline:\n\n"
            "{outline}\n\nUse the following references:\n\n<Documents>\n{docs}\n<Documents>",
        ),
        ("user", "Write the full blog section for the {section} section."),
    ]
)


async def retrieve(inputs: dict):
    docs = await retriever.ainvoke(inputs["topic"] + ": " + inputs["section"])
    formatted = "\n".join(
        [
            f'<Document/>\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )
    return {"docs": formatted, **inputs}


section_writer = (
    retrieve
    | section_writer_prompt
    | long_context_llm.with_structured_output(BlogSection)
)

In [ ]:
sections = await section_writer.abatch(
    [
        {
            "outline": refined_outline.as_str,
            "section": section.section_title,
            "topic": title,
        }
        for section in refined_outline.sections
    ]
)

In [ ]:
len(sections)

12

In [ ]:
draft = "\n\n".join([section.as_str for section in sections])

In [ ]:
from IPython.display import Markdown

# We will down-header the sections to create less confusion in this notebook
Markdown(section.as_str)

## Visibility and Reach

Consistent posting on social media platforms like LinkedIn significantly enhances your visibility and reach. The more frequently you post, the more opportunities you create for your content to be seen by a broader audience. This is not just about the sheer number of posts but the strategic timing and relevance of your content. Platforms like LinkedIn use sophisticated algorithms to determine which posts to display prominently in users' feeds. Here's a closer look at how these algorithms work and the future opportunities that increased visibility can bring:

### Algorithm Prioritization

Social media algorithms prioritize content based on visibility and engagement. When you post consistently, you increase your chances of appearing in your followers' feeds. These algorithms favor content that garners more interactions—likes, comments, and shares. This means that the more engaging your content is, the more likely it is to be seen by a wider audience. The algorithm recognizes patterns of high engagement and boosts your content's visibility, leading to even more interactions.

### Future Opportunities

Increased visibility on social media can open doors to numerous future opportunities. When more people see your content, you're more likely to attract attention from potential collaborators, clients, and industry leaders. This can lead to meaningful connections and collaborations that might not have been possible otherwise. Over time, this enhanced visibility can translate into tangible benefits such as speaking engagements, consulting offers, and job opportunities. By maintaining a consistent presence on social media, you position yourself as an active and influential member of your professional community, which can have long-term positive impacts on your career.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
writer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert blog author. Write the complete blog article on {topic} using the following section drafts:\n\n"
            "{draft}\n\nStrictly follow the blog outline guidelines.",
        ),
        (
            "user",
            'Write the complete blog article using markdown format.'
        ),
    ]
)

writer = writer_prompt | long_context_llm | StrOutputParser()

In [ ]:
for tok in writer.stream({"topic": title, "draft": draft}):
    print(tok, end="")

# Social Media Benefits Explained

## Introduction

In today's interconnected world, social media has transcended its original role as a platform for personal sharing and evolved into a powerful tool for professional growth, brand building, and community engagement. While the allure of follower counts and likes can be tempting, it's essential to delve deeper into the more substantial benefits that social media offers. Understanding these benefits can significantly enhance your strategy and help you make the most of your online presence.

This blog will explore various dimensions of social media benefits, from increased visibility and engagement to professional development and SEO advantages. By examining these aspects, we aim to provide a comprehensive understanding of how consistent and strategic social media posting can lead to meaningful opportunities and long-term success. So, let’s dive in and explore the profound impacts of social media beyond the superficial metrics of followers

In [ ]:
d = {
    "a": "hi",
    "b": "hello",
    "c": "bye"
}

print({**d, "c": "bye again"})

{'a': 'hi', 'b': 'hello', 'c': 'bye again'}
